# 1. Dataframe Preprocessing

In [ ]:
import pandas as pd
!pip3 install recordlinkage
import recordlinkage as rl
from recordlinkage.index import Block
from recordlinkage.preprocessing import clean
from recordlinkage.preprocessing import phonetic

# Regular expression operations
import re

     |████████████████████████████████| 952kB 5.1MB/s 
     |████████████████████████████████| 92kB 6.4MB/s 


In [ ]:
# mount drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# parse file
root = "drive/My Drive/Colab Notebooks"
df = pd.read_excel(root + "/current_master_3_21_2021.xlsx")

In [ ]:
for col in df.columns.unique():
  print(col)

unique_id
source
Census.Year
State.Province
County
Place
unsure_ids
no_ids
Household.Joint.ID
Joint.ID.for.Matched.Records
Last.Name
First.Name
CalculatedBirthYear
Age
Sex
Color..Race.or.Ethnicity
lat
long
address
MARITAL
WARD
ROLL.or.Sheet.
PROFESSION
Notable
Street.Address
PLACEOFBIRTH
RELIGION
NOTE.these.only.apply.to.narrative.answers
LIVING.W.MALE.FAMILY.
LIVING.W.FEMALE.FAMILY.
LIVING.W.MALE.NONFAMILY.
LIVING.W.FEMALE.NONFAMILY.
Cannot.Read
Cannot.Write
Sick
Relation.to.Head.of.House
Year.of.Immigration.to.Canada.if.an.Immigrant
Date.of.Death
Cause.of.Death
Rank_Military
Enlistment.Date
Enlistment.Place
Census.record.ID
Location.ID
Family.ID
Birth.Month
Birth.Day
Employment.History
Months.school
Earnings
English
French
Naturalization.Year
Mother.Tongue
Birthplace.Type
Race
Nationality
Has.Annote
Page
Line
Whitem
Colouredm
Colouredf
Totalm
Totalf
Childrenm
Childrenf
Schoolm
Schoolf
Notes.Other.Information
Last.Name.MATCH
First.Name.Match
Census.Year.Match
Total.of.Matches
Father.s

In [ ]:
# rename columns
df = df.rename(columns = {# Personal Information
                                       'Last.Name': 'last_name',
                                       'First.Name': 'first_name', 
                                       # Places or Geography
                                       'State.Province': 'state_or_province', 
                                       'County': 'county', 
                                       'Place': 'place', 
                                       'WARD': 'ward', 
                                       'Street.Address': 'street', 
                                       'PLACEOFBIRTH': 'place_of_birth',
                                       'ROLL.or.Sheet.': 'roll_or_sheet',
                                       # Years
                                       'Census.Year': 'census_year', 
                                       'CalculatedBirthYear': 'calculated_birth_year', 
                                       # Personal information
                                       'Sex': 'sex', 
                                       'Color..Race.or.Ethnicity': 'race',
                                       'MARITAL': 'marital_status',
                                       'PROFESSION': 'profession',
                                       'Notable': 'notable',
                                       'RELIGION': 'religion',
                          'Joint.ID.for.Matched.Records': 'Joint ID for Matched Records'
                          })

In [ ]:
# master info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59519 entries, 0 to 59518
Columns: 104 entries, unique_id to Whitef
dtypes: float64(43), int64(1), object(60)
memory usage: 47.2+ MB


In [ ]:
# chosen columns
chosen_columns = ['last_name', 
                  'first_name',
                  'state_or_province',
                  'county',
                  'place',
                  'ward',
                  'street',
                  'place_of_birth',
                  'roll_or_sheet',
                  'lat',
                  'long',
                  'census_year',
                  'calculated_birth_year',
                  'sex',
                  'race',
                  'marital_status',
                  'profession',
                  'religion',
                  'unique_id',
                  'phonetic_name']

# 2. Standardization/normalization

In [ ]:
import numpy as np

In [ ]:
#name, phonetic
column_to_parse = ['last_name', 'first_name']

for c in column_to_parse:
  df[c] = df.apply(lambda row: str(row[c]).lower(), axis = 1)

df["phonetic_name"] = phonetic(df['last_name'], 'soundex')+phonetic(df['first_name'], 'soundex')

In [ ]:
# race
def transform_race(row):    
    
    if pd.notnull(row['race']):
        row['race'] = row['race'].lower()
        row['race'] = row['race'].replace('\xa0', '')
        row['race'] = row['race'].replace('“', '')
        row['race'] = row['race'].replace('”', '')
    
    race_dict = {'mulatto(blackandwhite)': 'MIXED',
                 'm(wonancestry.com)': 'MIXED',
                 'mulatto': 'MIXED',
                 'mullato': 'MIXED',
                 'm': 'MIXED',
                 'm(winancestry.com)': 'MIXED',
                 'black': 'BLACK',
                 'b': 'BLACK',
                 'blk': 'BLACK',
                 'brown': 'BLACK',
                 'african': 'BLACK',
                 'dark': 'BLACK',
                 'drk': 'BLACK',
                 'african (black)': 'BLACK',
                 'negro': 'BLACK',
                 'blacj': 'BLACK', 
                 'bkj': 'BLACK', 
                 'light': 'WHITE',
                 'white': 'WHITE',
                 'w': 'WHITE',
                 '[w]': 'WHITE',
                 'white': 'WHITE',
                 'seems to be white': 'WHITE',
                 'white in black household': 'WHITE',
                 'white but passing': 'WHITE',
                 'ancestrysaysw': 'WHITE',
                 'swarthy': 'BLACK',
                 'd. brown': 'BLACK',
                 'col.d': 'BLACK',
                 'col\'d': 'BLACK',
                 }
    if row['race'] in race_dict.keys():
      return race_dict[row['race']]
    elif pd.notnull(row['race']):
      return 'OTHERS'
    else:
     return row['race']

df['race'] = df.apply(lambda row: transform_race(row), axis = 1)

In [ ]:
df['state_or_province'].unique()

array([nan, 'New York', 'Massachusetts', 'Michigan', 'Pennsylvania',
       'Missouri', 'Maine', 'Ohio', 'Illinois', 'Rhode Island',
       'Virginia', 'Wisconsin', 'Canada West', 'Ontario', 'Connecticut',
       'California', 'New Jersey', 'Minnesota', 'Delaware', 'Maryland',
       'Indiana', 'Texas', 'Washington DC', 'Iowa', 'Kansas', 'Vermont',
       'Oregon', 'Louisiana', 'British Columbia', 'Mississippi',
       'Oklahoma', 'Nebraska', 'Florida', 'Kentucky', 'Colorado',
       'Washington', 'Arkansas', 'West Virginia', 'NB', 'Tennessee',
       'South Carolina', 'Montana', 'Alabama', 'Alaska', 'Virgin Islands',
       'Georgia', 'North Carolina', 'New Mexico', 'MIL: Usnavy',
       'New Hampshire', 'Puerto Rico', 'North Dakota', 'Arizona', 'PANA',
       'South Dakota', 'Wyoming', 'Idaho', 'Nevada', 'Hawaii',
       'MIL: Philippines', 'MIL: Maryland', 'MIL: Massachusetts', 'Utah',
       'Massachussetts', 'MIL: Southcarolina', 'MIL: Cuba', 'ITER',
       'District of Columbia',

In [ ]:
# locations (nameparser, fuzzy matching)
def transform_state(row):    
    """
    This function uses a dictionary
    to covert historical or non-abbreviated states or provinces
    into abbreviated form. 
    An important notice is that Canada West will all be converted into ON (Ontario).
    Unknown values: PANA, ITER, MIL.
    """
    state_dict = {
                  'CO': 'Colorado',
                  'MA': 'Massachussetts',
                  'VA': 'Virginia',
                  'IL': 'Illinois',
                  'ME': 'Maine',
                  'NY': 'New York',
                  'MI': 'Michigan',
                  'WA': 'Washington',
                  'IA': 'Iowa',
                  }
    
    if row['state_or_province'] in state_dict.keys():
        return state_dict[row['state_or_province']]
    else:
        return row['state_or_province']

df['state_or_province'] = df.apply(lambda row: transform_state(row), axis = 1) 

In [ ]:
# dates
import datetime
def prc_year(row, year_col):
    """
    Processing birth year.
    If birth year is not a number, not in a date format, or is noncompliant,
    it will be converted to NaN.
    """
    
    # Helper functions
    def is_number(num):
        try:
            float(num)
            return True
        except ValueError:
            return False
    if (type(row[year_col]) is datetime.datetime):
        row[year_col] = row[year_col].strftime("%Y")
    non_compliant_values = ['-', 'F', '#VALUE!']
    if row[year_col] in non_compliant_values or pd.isnull(row[year_col]):
        return np.nan
    elif is_number(row[year_col]):
        return float(row[year_col])
    elif re.findall('\d{4}', row[year_col]):
        return min([float(i) for i in re.findall('\d{4}', row[year_col])])
    else:
        return np.nan

df['calculated_birth_year'] = df.apply(lambda row: prc_year(row, 'calculated_birth_year'),
                                       axis = 1)
df['census_year'] = df.apply(lambda row: prc_year(row, 'census_year'),
                             axis = 1)

In [ ]:
# clean place of birth
def transform_birthplace(row):    
    """
    This function uses a dictionary
    to covert places of birth to cleaner strings
    """
    if pd.notnull(row['place_of_birth']):
      row['place_of_birth'] = str(row['place_of_birth'])
      row['place_of_birth'] = row['place_of_birth'].lower()
      row['place_of_birth'] = row['place_of_birth'].replace('\xa0', '')
      row['place_of_birth'] = row['place_of_birth'].replace('“', '')
      row['place_of_birth'] = row['place_of_birth'].replace('”', '')
      row['place_of_birth'] = row['place_of_birth'].replace('(', '')
      row['place_of_birth'] = row['place_of_birth'].replace(')', '')
      row['place_of_birth'] = row['place_of_birth'].replace('[', '')
      row['place_of_birth'] = row['place_of_birth'].replace(']', '')
 
      return row['place_of_birth']

df['place_of_birth'] = df.apply(lambda row: transform_birthplace(row), axis = 1) 

In [ ]:
df['place_of_birth'].unique()

array(['usa', 'canada', 'maine', 'cw', 'can', 'ohio', 'michigan',
       'british columbia, canada', None, 'victoria bc', 'canadawest',
       'north carolina', 'uc', 'u.c.', 'new brunswick', 'mo', 'u. canada',
       'windsor', 'ontario', '24990', 'unitedkingdom,n.s.', 'kentucky',
       'indianna', 'detroit', 'northcarolina', 'chatham', 'texas',
       '44030', 'do meaning u canada upper canada', 'md', 'lc',
       'pennsylvania', 'virginia', 'arkansas', 'canadaw', 'minnesota',
       'indiana', 'kansas', '44010', 'mississippi', 'chathamcw',
       'missouri', 'wisconsin', 'amherstburg', 'united states',
       'new york', 'ireland', 'ny', '99999', 'us', 'u.s.', 'germany',
       'o ontario', 'washington', 'nj', 'lisa', 'new mexico', 'tennessee',
       'newyork', '21230', 'milwaukie', 'illinois', 'mexico', '21130',
       'eng', 'pa', 'il', 'ky', 'w', 'domeaningindiana', 'maryland',
       '43090', '24010', 'ma', 'mi', '21140', '31010', 'alaska', 'va',
       'sandwich', 'sweden', '

In [ ]:
# fill empty columns
columns_to_impute_with_unknown = ['state_or_province',
                                  'county',
                                  'place',
                                  'ward',
                                  'street',
                                  'place_of_birth',
                                  'sex',
                                  'race',
                                  'marital_status',
                                  'profession',
                                  'religion']

for i in columns_to_impute_with_unknown:
    df[i] = df[i].fillna('unknown')

columns_to_impute_with_zero = ['roll_or_sheet',
                               'lat',
                               'long',]

for i in columns_to_impute_with_zero:
    df[i] = df[i].fillna(0)

In [ ]:
# dealing with the 'unknowns'
chrs = 'abcdefghijklmnopqrstuvwxyz0123456789 '

location_columns = ['county', 'place', 'place_of_birth', 'state_or_province']

count = 0

for i, row in df.iterrows():
    if all([row[i] == 'unknown' for i in location_columns]):
      continue
    else:
      for l in location_columns:
        c = "".join(np.random.choice(list(chrs), 20))
        row[l] = c

# 3. Linkage

In [ ]:
# match all
df1 = df[chosen_columns]
df2 = df[chosen_columns]

# block on phoenetic full name
indexer = rl.Index()
indexer.add(Block('phonetic_name', 'phonetic_name'))
record_links = indexer.index(df1, df2)


In [ ]:
df[pd.isnull(df['Joint ID for Matched Records'])].shape


(52319, 105)

In [ ]:
# unmatched VS matched
# df_withmatch = df[pd.notnull(df['Joint ID for Matched Records'])]
# df_nomatch = df[pd.isnull(df['Joint ID for Matched Records'])]
# chosen_columns.append("phonetic_name")

# df_withmatch = df_withmatch[chosen_columns]
# df_nomatch = df_nomatch[chosen_columns]

# indexer = rl.Index()
# indexer.add(Block('phonetic_name', 'phonetic_name'))
# record_links = indexer.index(df_withmatch, df_nomatch)

In [ ]:
comparer = rl.Compare()
comparer.string('first_name', 'first_name', method = 'jarowinkler', threshold = 0.85, label = 'first_name')
comparer.string('last_name', 'last_name', method = 'jarowinkler', threshold = 0.85, label = 'last_name')
comparer.string('state_or_province', 'state_or_province', method = 'jarowinkler', threshold = 0.5, 
                label = 'state_or_province')
comparer.string('county', 'county', method = 'jarowinkler', threshold = 0.35, label = 'county')
comparer.string('place', 'place', method = 'jarowinkler', threshold = 0.35, label = 'place')
comparer.string('place_of_birth', 'place_of_birth', method = 'jarowinkler', threshold = 0.6, label = 'place_of_birth')
comparer.string('race', 'race', method = 'jarowinkler', threshold = 0.7, label = 'race')
comparer.string('sex', 'sex', method = 'jarowinkler', threshold = 0.7, label = 'sex')

comparer.numeric('calculated_birth_year', 'calculated_birth_year', 
                 method = 'gauss', 
                 offset = 1, 
                 scale = 1, 
                 label = 'calculated_birth_year')

<Compare>

Output Processing

In [ ]:
import sys
sys.setrecursionlimit(1000000)

In [ ]:
compare_vectors_rl = comparer.compute(record_links, df1, df2)

In [ ]:
result_rl = compare_vectors_rl[((compare_vectors_rl['first_name'] == 1.0) & 
                                (compare_vectors_rl['last_name'] == 1.0)) &
                               ((compare_vectors_rl['state_or_province'] == 1.0) |
                                (compare_vectors_rl['county'] == 1.0) |
                                (compare_vectors_rl['place'] == 1.0) |
                                (compare_vectors_rl['place_of_birth'] == 1.0)) &
                               (compare_vectors_rl['sex'] == 1.0) &
                               (compare_vectors_rl['race'] == 1.0) & 
                               ((compare_vectors_rl['calculated_birth_year'] > compare_vectors_rl['calculated_birth_year'].mean())
                               | (compare_vectors_rl['calculated_birth_year'] == np.nan))].\
                               reset_index()

result_rl = result_rl[result_rl['level_0'] != result_rl['level_1']].reset_index()
result_rl.drop('index', axis = 1, inplace = True)

In [ ]:
def create_indexid(row):
    return "".join(sorted([str(int(i)) for i in [row['level_0'], row['level_1']]]))

result_rl['indexid'] = result_rl.apply(lambda row: create_indexid(row), axis = 1)
result_rl = result_rl.drop_duplicates('indexid')

In [ ]:
# parse unsures into a dictionary
from collections import defaultdict
already_seen_pairs = defaultdict(set)

for id, list_unsures in list(zip(df.unique_id, df.unsure_ids)):
  if list_unsures[1:-1] != "":
    already_seen_pairs[id] = set(list_unsures[1:-1].split(", "))
  if id == 41810:
    print(list_unsures[1:-1])

for id, list_nos in list(zip(df.unique_id, df.no_ids)):
  if list_nos[1:-1] != "":
    already_seen_pairs[id] = set(list_nos[1:-1].split(", "))

42190, 41360


In [ ]:
df_result_rl.reset_index(inplace = True)
df_result_rl = df_result_rl.drop('index', axis = 1)
df_result_rl.head()
df_result_rl.shape

(3014, 105)

In [ ]:
num_list = []
for i in range(int(df_result_rl.shape[0]/2)):
    num_list.append(i)
    num_list.append(i)
df_result_rl['dup_pair'] = pd.Series(num_list)

In [ ]:
col_order = ['dup_pair']
col_order.extend(df.columns.tolist())
df_result_rl = df_result_rl[col_order]
output = root + "/newmatches-3-17-21.xlsx"
df_result_rl.to_excel(output)

In [ ]:
df_result_rl.head()

,dup_pair,unique_id,source,census_year,state_or_province,county,place,unsure_ids,no_ids,Household.Joint.ID,Joint ID for Matched Records,last_name,first_name,calculated_birth_year,Age,sex,race,lat,long,address,marital_status,ward,roll_or_sheet,profession,notable,street,place_of_birth,religion,NOTE.these.only.apply.to.narrative.answers,LIVING.W.MALE.FAMILY.,LIVING.W.FEMALE.FAMILY.,LIVING.W.MALE.NONFAMILY.,LIVING.W.FEMALE.NONFAMILY.,Cannot.Read,Cannot.Write,Sick,Relation.to.Head.of.House,Year.of.Immigration.to.Canada.if.an.Immigrant,Date.of.Death,Cause.of.Death,...,Childrenm,Childrenf,Schoolm,Schoolf,Notes.Other.Information,Last.Name.MATCH,First.Name.Match,Census.Year.Match,Total.of.Matches,Father.s.Name,Mother.s.Name,Spouse,City,MiddleName,Residence.Place,Death.Place,Family.Number,Attended.School,House.Number,Father.s.Birth.Place,Mother.s.Birth.Place,Immigration.Year,Able.to.read,Able.to.Write,REGIMENT,ORDER.NUMBER,Regt.,Application,Full.Name,Co.,Height,Complex.,Father.of.Foreign.Birth,Mother.of.Foreign.Birth,Township,Sheet.Number,Noonan.Page.Number,Employment,Whitef,phonetic_name
0,0,48688.0,1880 IPUMS 100% sample,1880.0,unknown,unknown,unknown,"[49221, 49093]",[],NaN,NaN,williams,george,1880.0,0,M,BLACK,0.000000,0.000000,4,1,0,0,ATHOME,0,1,usa,1,1,NaN,4.0,1.0,4.0,0,0,99.0,99,1.0,NaN,3,...,NaN,NaN,NaN,NaN,|||,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,W452G620
1,0,49094.0,13040,1910.0,Illinois,Will,Joliet,"[1756, 49093, 49221, 48632, 49093]",[48632],NaN,NaN,williams,george,1878.0,32,M,BLACK,41.419406,-87.999475,"JOLIET,WILL,IL",unknown,unknown,0,,YCHECK,,unknown,unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,|||,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,W452G620
2,1,49093.0,4475,1900.0,Missouri,Jackson,Kansascity,"[1756, 49094, 49221, 49094, 48688]",[49219],NaN,NaN,williams,george,1878.0,22,M,BLACK,39.035409,-94.358275,"KANSASCITY,JACKSON,MO",unknown,unknown,0,,YCHECK,,unknown,unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,|||,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,W452G620
3,1,49094.0,13040,1910.0,Illinois,Will,Joliet,"[1756, 49093, 49221, 48632, 49093]",[48632],NaN,NaN,williams,george,1878.0,32,M,BLACK,41.419406,-87.999475,"JOLIET,WILL,IL",unknown,unknown,0,,YCHECK,,unknown,unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,|||,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,W452G620
4,2,3468.0,1900 IPUMS 5% sample,1900.0,unknown,unknown,unknown,[],[],NaN,NaN,artist,elige,1857.0,43,M,BLACK,0.000000,0.000000,5,19,0,0,Gardener,0,10,canada,1,1,0.0,1.0,20.0,5.0,3,0,19.0,15,0.0,0,1,...,NaN,NaN,NaN,NaN,|||,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A632E420
